### Tasa de desempleo
[Link](https://totoro.banrep.gov.co/analytics/saw.dll?Portal&PortalPath=%2Fshared%2FDashboards_T%2FD_Estad%C3%ADsticas%2FEstad%C3%ADsticas&NQUser=publico&NQPassword=publico123&lang=es&page=Actividad%20econ%C3%B3mica,%20mercado%20laboral%20y%20cuentas%20financieras&pagina=Ocupaci%C3%B3n%20y%20desempleo)

In [23]:
from selenium.webdriver.common.by import By
import pandas as pd
import time
import sys
import os

cwd = os.getcwd()
cwd = cwd.replace("Notebooks", "Scripts")
sys.path.insert(0, cwd.replace("\\\\", "\\"))
import base as bs

In [24]:
def extraer(start_date: str, end_date: str):
    driver = bs.ejecutar_driver('https://totoro.banrep.gov.co/analytics/saw.dll?Portal&PortalPath=%2Fshared%2FDashboards_T%2FD_Estad%C3%ADsticas%2FEstad%C3%ADsticas&NQUser=publico&NQPassword=publico123&lang=es&page=Actividad%20econ%C3%B3mica,%20mercado%20laboral%20y%20cuentas%20financieras&pagina=Ocupaci%C3%B3n%20y%20desempleo')
    frame = driver.find_element(By.XPATH, './/iframe[@id="frame_dashboard"]')
    driver.switch_to.frame(frame)
    driver.find_element(By.XPATH, '//div[@title="Ocupación y desempleo"]').click()
    driver.find_elements(By.XPATH, '//span[@id="shielddy"]//a')[2].click()
    
    time.sleep(10)
    driver.quit()
    
    fuente_archivo = bs.obtener_nombre_descarga('/Users/'+os.getlogin()+'/Downloads')
    
    df = pd.read_excel(fuente_archivo, header=5)
    df.drop(df.tail(15).index, inplace=True)
    df['date'] = df['Año-Mes (AAAA-MM)'].map(lambda x: '01'+x[4:]+'-'+x[:4])
    
    aux_1 = start_date.replace(start_date[:2], '01')
    aux_2 = end_date.replace(end_date[:2], '01')

    try:
        start = df[df['date'] == aux_1].index[0]
    except IndexError:
        start = 0

    try:
        end = df[df['date'] == aux_2].index[0]
    except IndexError:
        end = len(df) - 1
        
    df['Granularidad'] = 'Mensual'
    df['Indicador'] = 'Tasa de Desempleo'
    df['Unidad'] = '%'
    df.rename(columns={'Tasa de desempleo (%)': 'Valor'}, inplace=True)

    df = df.loc[start:end, ['date', 'Granularidad', 'Indicador',
                            'Unidad', 'Valor']].reset_index().drop(['index'], axis=1)
    
    os.remove(fuente_archivo)
    
    return df

In [25]:
print(extraer('01-01-1990', '01-01-2040'))

           date Granularidad          Indicador Unidad      Valor
0    01-03-2023      Mensual  Tasa de Desempleo      %  10.032544
1    01-02-2023      Mensual  Tasa de Desempleo      %  11.353126
2    01-01-2023      Mensual  Tasa de Desempleo      %  13.704360
3    01-12-2022      Mensual  Tasa de Desempleo      %  10.272679
4    01-11-2022      Mensual  Tasa de Desempleo      %   9.500758
..          ...          ...                ...    ...        ...
262  01-05-2001      Mensual  Tasa de Desempleo      %  14.035800
263  01-04-2001      Mensual  Tasa de Desempleo      %  14.515100
264  01-03-2001      Mensual  Tasa de Desempleo      %  15.811900
265  01-02-2001      Mensual  Tasa de Desempleo      %  17.434200
266  01-01-2001      Mensual  Tasa de Desempleo      %  16.622300

[267 rows x 5 columns]
